In [ ]:
from sklearn.preprocessing import (
    MaxAbsScaler,
    MinMaxScaler,
    Normalizer,
    PowerTransformer,
    QuantileTransformer,
    RobustScaler,
    StandardScaler,
    minmax_scale,
)
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.ensemble import IsolationForest
import optuna
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader

d:\GitHub repos\ADL\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def fold_to_dataloader_tensor(train_x, test_x, train_y, test_y, batch_size=64, device=device):
    train_dataset = TensorDataset(
        torch.tensor(train_x.values,dtype=torch.float32).to(device), 
        torch.tensor(train_y.values,dtype=torch.float32).to(device))
    val_dataset = TensorDataset(
        torch.tensor(test_x.values,dtype=torch.float32).to(device), 
        torch.tensor(test_y.values,dtype=torch.float32).to(device))

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=True, drop_last=True)
    return train_loader, val_loader 


NameError: name 'device' is not defined

In [ ]:
random_state = 42 #! ABSOLUTELY DO NOT CHANGE THIS VALUE AS IT WILL CHANGE THE TEST/HOLDOUT SET
raw_dataset = pd.read_csv("./data/processed_data_OHE.csv") #data has X and Y
X = raw_dataset.drop(columns=["DR"])
Y = pd.DataFrame(raw_dataset["DR"])
# Slice your data


X_FOR_FOLDS, X_FINAL_TEST, Y_FOR_FOLDS, Y_FINAL_TEST = train_test_split(X, Y, test_size=0.1, random_state=random_state, stratify=Y)
HOLDOUT_DATA = pd.concat([X_FINAL_TEST, Y_FINAL_TEST], axis=1)
FOLDS_DATA = pd.concat([X_FOR_FOLDS, Y_FOR_FOLDS], axis=1)

In [ ]:
def FOLDS_GENERATOR(dataset, n_splits=5, random_state=None, oversampler=None, noise=None,
                     OD_majority=None, OD_minority=None, scaler=None):
    kF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    kFolds_list = []

    # Convert column names to strings to ensure compatibility
    df = dataset.copy()
    X = df.drop(columns=["DR"])
    Y = pd.DataFrame(df["DR"])

    for fold, (train_idx, test_idx) in enumerate(kF.split(X, Y)):
        # Split the data into training and testing sets for this fold
        train = pd.concat([X.iloc[train_idx], Y.iloc[train_idx]], axis=1)
        test = pd.concat([X.iloc[test_idx], Y.iloc[test_idx]], axis=1)
        
        # Apply P to X_train and X_test, passing Y_train to P for class info
        X_train_processed, X_test_processed = Preprocessor(train, test,
                                                OD_majority=OD_majority,
                                                OD_minority=OD_minority,
                                                scaler=scaler)
        # Append the processed fold to the list
        # kFolds_list.append((X_train_processed, X_test_processed))  # Append imputed DataFrames
        kFolds_list.append((X_train_processed.drop(columns=["DR"]),
                            X_test_processed.drop(columns=["DR"]),
                            X_train_processed[["DR"]],
                            X_test_processed[["DR"]]))

        print(f"Fold: {fold+1}, Train: {X_train_processed.shape}, Test: {X_test_processed.shape}")
    return kFolds_list

if outlier_detection:
    do OD AND REMOVAL
    
oversample= Smotenc
oversample + original training
synthetic = synthesizer(oversample + original training)

print similarity scores between original and synthetic data

scaler = StandardScaler()

save training and test data for each fold, automatic numbering convention